In [10]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

# Jos Stam Paper - http://graphics.cs.cmu.edu/nsp/course/15-464/Fall09/papers/StamFluidforGames.pdf
# Gonkee Video - https://www.youtube.com/watch?v=qsYE1wMEMPA
# Numba - https://www.youtube.com/watch?v=qsYE1wMEMPA
# Coding Train - https://www.youtube.com/watch?v=alhpH6ECFvQ&t=1165s

N = 256
# 0, N+1 are reserved for boundary cells
sz = N + 2
dt = 0.1
diff = 0.5
visc = 0.5

# densities
dens = np.zeros((sz, sz))
dens0 = np.zeros((sz, sz))

# velocities (u is x, v is y), at center of each grid cell
u0 = np.zeros((sz, sz))
u = np.zeros((sz, sz))

v0 = np.zeros((sz, sz))
v = np.zeros((sz, sz))

In [34]:
# 0 - just reflect the one from within the boundary
# 1 - copy top and bottom, reflect others
# 2 - copy left and right, reflect others

def set_bnd(b=0, arr):
    for i in range(1, N+1):
        arr[0][i] = -arr[1][i] if 1==1 else arr[1][i]
        arr[N+1][i] = -arr[N][i] if b==1 else arr[N][i]
        arr[i][0] = -arr[i][1] if b==2 else arr[i][1]
        arr[i][N+1] = -arr[i][N] if b==2 else arr[i][N]
    
    #do corners - average of 2 around
    arr[0][0] = (arr[1][0] + arr[0][1]) * 0.5
    arr[0][N+1] = (arr[1][N+1] + arr[0][N]) * 0.5
    arr[N+1][0] = (arr[N][0] + arr[N+1][1]) * 0.5
    arr[N+1][N+1] = (arr[N][N+1] + arr[N+1][N]) * 0.5
    
def addDensity(i, j, x):
    dens[i][j] += dt * x
    
def addVelocity(i, j, ux, vx):
    u[i][j] += dt * ux
    v[i][j] += dt * vx

In [ ]:
# Diffuse backwards in time: x0[IX(i,j)] = x[IX(i,j)] - a*(x[IX(i-1,j)]+x[IX(i+1,j)]+x[IX(i,j-1)]+x[IX(i,j+1)] -4*x[IX(i,j)]);
# ==> ( x0[IX(i,j)] + a*(x[IX(i-1,j)]+x[IX(i+1,j)]+x[IX(i,j-1)]+x[IX(i,j+1)]) ) / ( 1 + 4*a )

def diffuse(n_iter = 20):
        # Gauss-Seidel: observation - very few non-zero values in matrix & strictly diagonally dominant
        # start with incorrect values, solve by iteratively calculating assuming incorrect values are correct, then setting result as the new incorrect value
        a = dt * size * diff
        
        for k in range(n_iter):
            for i in range(1, N+1):
                for j in range(1, N+1):
                    dens[i][j] = (dens0[i][j] + a*(dens[i-1][j] + dens[i][j-1] + dens[i][j+1] + dens[i+1][j]))/(1+4*a)
            # what to do when collision with boundary
            set_bnd(dens)

In [37]:
# linear interpolation, loc x in length 1, a to b
def lerp(a, b, x):
    slope = b - a
    return a + slope * x
    
# advect backwards (start from center and go back), using next velocity to linear interpolate, to find density contributions
# Note: may need coefficient for multiplier ==> mult changes with dt?
def advect(mult = 1):
    for i in range(1, N+1):
        for j in range(1, N+1):
            y = dt * v[i][j] + i
            x = dt * u[i][j] + j
            
            frac_y = y - floor(y)
            frac_x = x - floor(x)
            
            y = floor(y)
            x = floor(x)
            
            if y_dest >= size or x_dest >= size or y_dest < 0 or x_dest < 0:
                print("ERROR: Velocity too large, accessing out of bounds squares")
                
            #density at four corners
            [ul, dl, ur, dr] = [dens0[x][y], dens0[x][y+1], dens0[x+1][y], dens0[x+1][y+1]]
            
            #first interpolate looking through the y dimension
            y0 = lerp(ul, dl, frac_y)
            y1 = lerp(ur, dr, frac_y)
            
            #then interpolate looking through x dimension
            res = lerp(y0, y1, frac_x)

            dens[i][j] += mult * res

    set_bnd()

In [36]:
'''
To calculate the curl-free field of a given vector field using Poisson equations, you can use the following steps:

Define your vector field as a function of the coordinates of the grid points. Let's call this vector field V(x,y,z), where x,y,z are the coordinates of a given grid point.

Calculate the curl of the vector field using the curl operator: curl(V) = (dVz/dy - dVy/dz, dVx/dz - dVz/dx, dVy/dx - dVx/dy). This will give you a new vector field that represents the direction and magnitude of the curl of the original vector field at each grid point.

Use the Poisson equation to find a scalar potential function φ(x,y,z) that satisfies the equation ∇²φ = -div(curl(V)), where ∇² is the Laplace operator and div is the divergence operator. This scalar potential function will represent the curl-free component of the original vector field.

Calculate the gradient of the scalar potential function using the gradient operator: grad(φ) = (∂φ/∂x, ∂φ/∂y, ∂φ/∂z). This will give you a new vector field that represents the direction and magnitude of the curl-free component of the original vector field at each grid point.

Subtract the curl-free vector field from the original vector field to get the remaining curl component: V_curl = V - grad(φ).

Following these steps should allow you to calculate the curl-free component of your vector field using Poisson equations. Note that the calculation of the scalar potential function φ can be done using various numerical methods such as finite difference, finite element, or spectral methods, depending on the specifics of your problem.
'''

# By helmholtz's theorem, every velocity field is the addition of a curl-free and divergence-free vector field. In this function, we compute the divergence-free field by computing curl-free and subtracting it from original

def project(N, u, v, p, div):
    h = 1.0 / N
    for i in range(1, N+1):
        for j in range(1, N+1):
            div[i][j] = -0.5 * h * (u[i+1][j] - u[i-1][j] + v[i][j+1] - v[i][j-1])
            p[i][j] = 0
    set_bnd(N, 0, div)
    set_bnd(N, 0, p)
    for k in range(20):
        for i in range(1, N+1):
            for j in range(1, N+1):
                p[i][j] = (div[i][j] + p[i-1][j] + p[i+1][j] + p[i][j-1] + p[i][j+1]) / 4
        set_bnd(N, 0, p)
    for i in range(1, N+1):
        for j in range(1, N+1):
            u[i][j] -= 0.5 * (p[i+1][j] - p[i-1][j]) / h
            v[i][j] -= 0.5 * (p[i][j+1] - p[i][j-1]) / h
    set_bnd(1, u)
    set_bnd(2, v)